In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
from dark_emulator_public import dark_emulator
import os, sys, time, json
import matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.interpolate import InterpolatedUnivariateSpline as ius
from scipy.interpolate import interp2d, interp1d
from scipy.integrate import simps
from tqdm import tqdm
import hsc3x2pt

using dark_emulator at  /Users/sugiyamasunao/Documents/python_package/dark_emulator_public/dark_emulator/__init__.py


In [3]:
power_b1 = hsc3x2pt.power_b1_class()

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


In [33]:
def getFisherMat(dirname, power_b1):
    pk2cl_fid = hsc3x2pt.pk2cl_class(power_b1)
    pk2cl_fid.load_Cl_cache(os.path.join(dirname, 'fiducial'))
    
    x = hsc3x2pt.radial_bin_class(pk2cl_fid.get_all_galaxy_sample())
    
    cov = hsc3x2pt.covariance_class(x)
    cov.set_covariance_from_pk2cl(pk2cl_fid)
    c = cov.get_full_covariance()
    cc = hsc3x2pt.correlation_coeff(c)
    ic = np.linalg.inv(c)
    
    y = hsc3x2pt.signal_class(x)
    y.set_signal_from_pk2cl(pk2cl_fid)
    v_fid = y.get_signal()
    
    file_param = json.load(open(os.path.join(dirname, 'file_param.json'), 'r'), object_pairs_hook=od)

    v_list = []
    dvdp_list = []
    for fname, dp in file_param.items():
        pk2cl = hsc3x2pt.pk2cl_class(power_b1)
        pk2cl.load_Cl_cache(os.path.join(dirname, fname))
        y.set_signal_from_pk2cl(pk2cl)
        v = y.get_signal()
        v_list.append(v)
        dvdp = (v-v_fid)/dp
        dvdp_list.append(dvdp)
    dvdp_list = np.array(dvdp_list)
    
    F = np.dot(dvdp_list, np.dot(ic, dvdp_list.T))
    param_names = list(file_param.keys())
    return param_names,F

In [34]:
def skip_column(arr, i):
    if arr.ndim == 1:
        return np.hstack([arr[:i], arr[i+1:]])
    else:
        return np.vstack([arr[:i, :], arr[i+1:, :]])

def skipMat(mat, i):
    return skip_column(skip_column(mat, i).T, i).T

def getSigma(F, i):
    F11 = F[i,i]
    F12 = skip_column(F[i, :], i)
    F22 = skipMat(F, i)
    iF22 = np.linalg.inv(F22)

    isigma2 = F11 - np.dot(F12, np.dot(iF22, F12))
    sigma = 1.0/isigma2**0.5
    return sigma

# Year 1, single source bin analysis

In [36]:
%%time
dirname = './single_source_ClY1'
pnames, F = getFisherMat(dirname, power_b1)

CPU times: user 18.1 s, sys: 921 ms, total: 19 s
Wall time: 4.98 s


In [37]:
for i in range(13):
    sigma = getSigma(F, i)
    print(pnames[i], sigma)

omega_b 0.039067965586619346
omega_c 0.08725041644590159
Omega_de 0.06531512993236395
n_s 0.09434801893222415
ln10p10As 0.6027872131776162
b1lowz 0.2805740986879434
b1cmass1 0.2012227284421203
b1cmass2 0.21148498307778432
alphamaglowz 46.955828264095864
alphamagcmass1 3.566664369993745
alphamagcmass2 2.6923238913942926
dzph 0.2928971036637266
dm 0.35065637471566535


(12,)

In [27]:
iF22.shape

(12, 12)

In [24]:
F22.shape

(12, 12)

In [6]:
pk2cl_fid = hsc3x2pt.pk2cl_class(power_b1)
pk2cl_fid.load_Cl_cache(os.path.join(dirname, 'fiducial'))

In [7]:
x = hsc3x2pt.radial_bin_class(pk2cl_fid.get_all_galaxy_sample())

In [8]:
cov = hsc3x2pt.covariance_class(x)
cov.set_covariance_from_pk2cl(pk2cl_fid)
c = cov.get_full_covariance()
cc = hsc3x2pt.correlation_coeff(c)
ic = np.linalg.inv(c)

In [9]:
y = hsc3x2pt.signal_class(x)
y.set_signal_from_pk2cl(pk2cl_fid)
v_fid = y.get_signal()

In [10]:
F = getFisherMat(dirname, power_b1, ic, v_fid)